In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

'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'

In [ ]:
!pip install langchain_groq

In [7]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x119470d60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x119471a20>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content = "My name is Anusha and I am Data Scientist")])

AIMessage(content="Hello Anusha! It's nice to meet you.\n\nBeing a Data Scientist is fascinating work. What kind of projects are you working on these days? \n\nI'm eager to learn more about your work and maybe even how I can be helpful to you in the future. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 19, 'total_tokens': 83, 'completion_time': 0.116363636, 'prompt_time': 7.8079e-05, 'queue_time': 0.015475351, 'total_time': 0.116441715}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-835cf340-d0ba-49b4-816f-e3c4fd9bd3a5-0', usage_metadata={'input_tokens': 19, 'output_tokens': 64, 'total_tokens': 83})

In [9]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content = "Hi, My name is Anusha and I am data scientist"),
        AIMessage(content = "Hello Anusha! It's nice to meet you.\n\nBeing a Data Scientist is fascinating work. What kind of projects are you working on these days? \n\nI'm eager to learn more about your work and maybe even how I can be helpful to you in the future. 😊  \n\n"),
        HumanMessage(content = "Hey What's my name and what do i do?")
    ]
)

AIMessage(content="You are Anusha, and you are a data scientist!  \n\nIt seems like you're testing me.  I'm still under development and learning to remember information about people.  \n\nDo you have any other questions for me? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 104, 'total_tokens': 159, 'completion_time': 0.1, 'prompt_time': 0.005254933, 'queue_time': 0.007815457, 'total_time': 0.105254933}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3d23975d-146f-4dfa-bb04-a1513a8b8580-0', usage_metadata={'input_tokens': 104, 'output_tokens': 55, 'total_tokens': 159})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [13]:
config = {"configurable" : {"session_id" : "chat1"}}

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content = "My name is Anusha and I am Data Scientist")],
    config = config
)

In [21]:
response.content

"Hi Anusha!  \n\nIt's lovely to connect with another data scientist.  What area of data science are you most passionate about?  \n\nDo you have any exciting projects you're working on that you'd like to share? I'm always interested in learning about new developments in the field. 😊  \n\n\n\n"

In [22]:
with_message_history.invoke(
    [HumanMessage(content = "What is my name?")],
    config = config
)

AIMessage(content='Your name is Anusha.  😊 \n\nI remember from our previous conversations! \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 409, 'total_tokens': 431, 'completion_time': 0.04, 'prompt_time': 0.014007151, 'queue_time': 0.003157408999999998, 'total_time': 0.054007151}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e2941df-84fc-4bf1-92ef-1822bd3a687c-0', usage_metadata={'input_tokens': 409, 'output_tokens': 22, 'total_tokens': 431})

In [24]:
## changing the session id 

config1 = {"configurable" : {"session_id" : "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content = "Whats my name?")],
    config = config1
)

In [25]:
response.content

'As an AI, I have no memory of past conversations and no access to personal information about you. To know your name, I would need you to tell me! 😄\n'

In [26]:
response = with_message_history.invoke(
    [HumanMessage(content = "Hey My name is Anusha?")],
    config = config1
)

In [27]:
response.content

"Hi Anusha, it's nice to meet you! 😊\n\nIs there anything I can help you with today?\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [30]:
chain.invoke({"messages" : [HumanMessage(content = "Hi My name is Anusha")]})

AIMessage(content="Hello Anusha, it's nice to meet you! \n\nI'm happy to help with any questions you have.  Just ask away! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 32, 'total_tokens': 68, 'completion_time': 0.065454545, 'prompt_time': 0.000314269, 'queue_time': 0.014096061, 'total_time': 0.065768814}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c0909074-5b12-490e-be9b-8c4f55737747-0', usage_metadata={'input_tokens': 32, 'output_tokens': 36, 'total_tokens': 68})

In [ ]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Anusha")],
    config=config
)

response

AIMessage(content="Hi Krish, it's nice to meet you!\n\nWhat can I do for you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 14, 'total_tokens': 39, 'completion_time': 0.045454545, 'prompt_time': 8.496e-05, 'queue_time': 0.014206389, 'total_time': 0.045539505}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f0ce7e82-b645-4a18-85bf-12d30399ba87-0', usage_metadata={'input_tokens': 14, 'output_tokens': 25, 'total_tokens': 39})

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish.  😄  \n\nI remember that you told me earlier!\n'

In [33]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [34]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Anusha")],"language":"Hindi"})
response.content

'Namaste Anusha! 😊\n\nमुझे आपकी मदद करने में खुशी होगी। आपके कोई सवाल हैं?  \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [36]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [38]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Anusha")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते अनुषा!  \n\nआप कैसे हैं?  क्या मैं आपकी कोई मदद कर सकता हूँ? \n'

In [39]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Whats my name?")],"language":"Hindi"},
    config=config
)
repsonse.content

'आपका नाम अनुषा है। 😊  \n'